### Make a simple model

In [1]:
import category_encoders as ce
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

df = sns.load_dataset('titanic')
X = df[['sex', 'age', 'pclass']]
y = df['survived']

pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(), 
    RandomForestClassifier(n_estimators=50, n_jobs=-1, oob_score=True))

pipeline.fit(X, y)
pipeline.named_steps['randomforestclassifier'].oob_score_

0.8103254769921436

### Save it to a file

In [2]:
from joblib import dump, load
dump(pipeline, 'pipeline.joblib') 

['pipeline.joblib']

In [3]:
!ls -lh pipeline.joblib

-rw-r--r--  1 ryanherr  staff   783K Apr 12 21:15 pipeline.joblib


### Delete it from memory

In [4]:
del pipeline
pipeline.named_steps['randomforestclassifier'].oob_score_

NameError: name 'pipeline' is not defined

### Load it from the file

In [5]:
pipeline = load('pipeline.joblib') 
pipeline.named_steps['randomforestclassifier'].oob_score_

0.8103254769921436

### Make predictions for new observations

In [6]:
observations = {'columns': ['sex','age','pclass'], 
                'data': [['male',30,3], ['female',None,1]]}

In [7]:
pd.DataFrame(columns=observations['columns'], data=observations['data'])

,sex,age,pclass
0,male,30.0,3
1,female,NaN,1


In [8]:
pd.DataFrame(**observations)

,sex,age,pclass
0,male,30.0,3
1,female,NaN,1


In [9]:
X = pd.DataFrame(**observations)
pipeline.predict(X).tolist()

[0, 1]

### Put this code in a Flask app and run it

```python
from flask import Flask, jsonify, request
from joblib import load
import pandas as pd

app = Flask(__name__)
pipeline = load('pipeline.joblib')

@app.route('/predict', methods=['POST'])
def predict():
    X = pd.DataFrame(**request.get_json(force=True))
    y_pred = pipeline.predict(X)
    return jsonify(y_pred.tolist())
```

### Send and receive JSON from your Flask app

In [10]:
import requests
url = 'http://127.0.0.1:5000/predict'
data = '{"columns":["sex","age","pclass"],"data":[["male",30,3],["female",NaN,1]]}'
response = requests.post(url, data)

In [11]:
response

<Response [200]>

In [12]:
response.json()

[0, 1]